#EXERCISE 1

I USED NEURAL NETS HERE.
AND YES, THIS IS BETTER THAN BIGRAM MODEL.


In [ ]:
import torch
words = open("names.txt","r").read().splitlines()
chars = sorted(list(set('.'.join(words))))

In [ ]:
stoi={s:i for i,s in enumerate(chars)}
stoi['.'] = 0
itos={i:s for s,i in stoi.items()}


In [ ]:
#created bitoi, itobi for convenience
xs = []
ys = []
b =[]
for ch in chars:
  for char in chars:
    b.append((ch,char))

bi = sorted(list(set(b)))
bitoi = {(ch1,ch2):i for i, (ch1,ch2) in enumerate(bi)}
itobi = {i:s for s,i in bitoi.items()}

for w in words:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        xs.append(bitoi[(ch1, ch2)])
        ys.append(stoi[ch3])


In [ ]:
xs = torch.tensor(xs)
ys = torch.tensor(ys)
print(xs)
print(ys)

tensor([  5, 148, 364,  ..., 727, 701, 726])
tensor([13, 13,  1,  ..., 26, 24,  0])


In [ ]:
import torch.nn.functional as F
xenc = F.one_hot(xs, num_classes =729).float()
print(xenc)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 1., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 1., 0., 0.]])


In [ ]:
num = xs.nelement()
print(num)

196113


In [ ]:
g= torch.Generator().manual_seed(2147483652)
W = torch.randn((729,27),generator =g, requires_grad = True)

In [ ]:
#loss calculation
for k in range(10):

  xenc = F.one_hot(xs, num_classes=729).float()
  logits = xenc @ W
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdims=True)
  #print(probs.shape)
  loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
  print(loss.item())

  W.grad = None
  loss.backward()

  W.data += -100 * W.grad

2.5238139629364014
2.514756679534912
2.5060904026031494
2.497786045074463
2.489819288253784
2.4821698665618896
2.474818706512451
2.4677488803863525
2.4609439373016357
2.4543895721435547


In [ ]:
#OUTPUT
for i in range(30):
 out = []
 ix = 0

 while True:
  xenc = F.one_hot(torch.tensor([ix]), num_classes =729).float()
  logits = xenc @ W
  counts = logits.exp()
  p = counts / counts.sum(1,keepdims = True)
  #print(p.shape)
  var = list(itobi[ix])
  ixo = stoi[var[1]]
  ix = torch.multinomial(p, num_samples=1, replacement=True, generator = g).item()
  out.append(itos[ix])
  ixn = ix
  if ix==0:
    break
  ix = bitoi[(itos[ixo],itos[ixn])]
 print(''.join(out))


eeessrm.
cacalin.
ma.
grvplxwpnuaczclqsanalistt.
obbjbcnwmtvgel.
cyson.
glhnawubvznian.
pcltintzllkbkzlgkmya.
hadjslzbvlnysutlaznfbdjkpm.
gry.
na.
miwwsxev.
per.
cooxamipohvvyleigbypswhckbpodnqnzisswvyzlkhymcuo.
puzmxprxlscgh.
brsnirybkbxaudrek.
maj.
lijmon.
bribaxjnbllnyla.
bgknvxhhanna.
mir.
caklmifbtfyqegyp.
chhhadecbx.
pedyel.
on.
vi.
hashacjkrsfomaashadelzqqvjyueilqpfuckkjisthbbn.
p.
izahrrossco.
ivyyleigalfkfdbfvmhmwgjcvfrjkglhbicybsxcmqrlia.


# EXERCISE 2 (TRIGRAM)

loss function to calculate loss seperately for diff datasets

In [ ]:
def loss_fn(xs,ys):
  num = xs.nelement()
  xenc = F.one_hot(xs, num_classes=729).float()
  logits = xenc @ W
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdims=True)
  loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
  #print(loss.item())

  W.grad = None
  loss.backward()

  W.data += -50 * W.grad
  return loss.item()

seperating dataset

In [ ]:
import random
data = list(zip(xs, ys))
torch.manual_seed(42)
random.shuffle(data)

split1 = int(0.8 * len(data))
split2 = int(0.9 * len(data))

train_data = data[:split1]
dev_data = data[split1:split2]
test_data = data[split2:]

#training set
xs_train, ys_train = [], []
for x, y in train_data:
    xs_train.append(x)
    ys_train.append(y)

xs_train = torch.tensor(xs_train)
ys_train = torch.tensor(ys_train)

#dev set
xs_dev, ys_dev = [], []
for x, y in dev_data:
    xs_dev.append(x)
    ys_dev.append(y)

xs_dev = torch.tensor(xs_dev)
ys_dev = torch.tensor(ys_dev)

#test set
xs_test, ys_test = [], []
for x, y in test_data:
    xs_test.append(x)
    ys_test.append(y)

xs_test = torch.tensor(xs_test)
ys_test = torch.tensor(ys_test)


In [ ]:
train_loss = loss_fn(xs_train, ys_train)
dev_loss = loss_fn(xs_dev, ys_dev)
test_loss = loss_fn(xs_test, ys_test)
print(train_loss)
print(dev_loss)
print(test_loss)

2.449626922607422
2.4415647983551025
2.433560609817505


Hence, we can see that all are almost equal and also loss of dev set and test set are slightly lower than that of training set for this trigram model

# EXERCISE 2 (BIGRAM)

# EXERCISE 3

loss function to calculate loss seperately for diff datasets by varyling l for regularization

In [ ]:
def loss_fun(xs,ys,l):
  num = xs.nelement()
  xenc = F.one_hot(xs, num_classes=729).float()
  logits = xenc @ W
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdims=True)
  loss = -probs[torch.arange(num), ys].log().mean() + l*(W**2).mean()
  #print(loss.item())

  W.grad = None
  loss.backward()

  #W.data += -50 * W.grad
  return loss.item()

to observe loss of training set

In [ ]:
a = 0.0000001
for i in range(8):
  train_loss = loss_fun(xs_train, ys_train, a)
  print(train_loss)
  a*=10

2.39216685295105
2.392167806625366
2.392176389694214
2.3922626972198486
2.3931264877319336
2.4017629623413086
2.488129138946533
3.3517889976501465


to observe loss of dev set

In [ ]:
l = 0.0000001
for i in range(8):
 #print("trial :",i)
 print("l = ",l)
 dev_loss = loss_fun(xs_dev, ys_dev,l)
 print(dev_loss)
 l*=10

l =  1e-07
2.4240174293518066
l =  1e-06
2.4191877841949463
l =  9.999999999999999e-06
2.414562225341797
l =  9.999999999999999e-05
2.4101808071136475
l =  0.001
2.406733751296997
l =  0.01
2.4113123416900635
l =  0.1
2.4950225353240967
l =  1.0
3.368551731109619


And as we increase the regularization strength while finding train_loss and dev_loss, train_loss increases continuously whereas dev_loss decreases at first and increases again.

As we can see from the above loop, when l=0.001 dev_loss attains its minimum value

In [ ]:
test_loss = loss_fun(xs_test, ys_test, 0.001)
print(test_loss)

2.8470847606658936


This is the test_loss with best regularization strength("l")

# EXERCISE 4

In both the following cells, I avoided using one-hot vectors.
The code ran succesfully and the runtime was very low compared to that using onehot vectors

The resut of using onehot vectors was just picking out selected rows from W. So, I replaced it by writing W[xs]

In [ ]:
#loss calculation
for k in range(10):
  logits = W[xs]
  counts = logits.exp()
  probs = counts / counts.sum(1, keepdims=True)
  #print(probs.shape)
  loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
  print(loss.item())

  W.grad = None
  loss.backward()

  W.data += -100 * W.grad

2.8564624786376953
2.826683759689331
2.799455165863037
2.7744333744049072
2.751333236694336
2.7299115657806396
2.709965705871582
2.69132137298584
2.6738343238830566
2.657383680343628


In [ ]:
#OUTPUT
for i in range(30):
 out = []
 ix = 0

 while True:
  logits = W[ix]
  counts = logits.exp()
  probs = counts / counts.sum(0,keepdims = True)
  var = list(itobi[ix])
  ixo = stoi[var[1]]
  ix = torch.multinomial(probs, num_samples=1, replacement=True, generator = g).item()
  out.append(itos[ix])
  ixn = ix
  if ix==0:
    break
  ix = bitoi[(itos[ixo],itos[ixn])]
 print(''.join(out))


jjbdjmon.
brebaxjnbllnyht.
bgknvxhhanna.
mip.
caklmgjbofimtlxp.
chhhael.
bx.
pedyel.
on.
vhlhjkrpclpusfomaashadeyzqqvjyueikqpfuckkjisthbbn.
p.
izaurdsrmnxzivyylaidalfkfdbfvmhmwgjcvfrjkglhbicybsxcmqrlia.
peohjrymxan.
vdklzkwtfqqkpsgpriarrlarikton.
phell.
romnyjftkhykgy.
caeorcfrtxrjp.
brdqaqpgeyscccaptkkjwzissczwsdntgomtui.
hael.
helanriabpzrgdcsqawbfnplzgkncnywu.
coyojdbs.
ygsqpkywuxyj.
zsyruqcvopfhvplrendael.
wqhkvttxiimdgdhhqhzrqljlin.
msgsnmg.
breisfnjvqdqschavwvenicattscofnctmvuumdjlmewfjtmggfufevsxhtzpnpjylmtwgbbddrxsleisjszkmz.
kainijwbqwskjyipfce.
uawvuuwftpjacxgsizodakprsyoady.
madefelazmon.
gwspvopipwdkurubc.


# EXERCISE 5

I used F.cross_entropy in this cell

It replaced many lines of code and made it simpler so maybe that's why we prefer this

As we can see below loss calculated from two methods is the same

In [ ]:
#loss calculation using F.cross_entropy
logits = W[xs]
cross_entropy_loss = F.cross_entropy(logits,ys) + 0.01*(W**2).mean()
print(cross_entropy_loss.item())

counts = logits.exp()
probs = counts / counts.sum(1, keepdims=True)
loss = -probs[torch.arange(num), ys].log().mean() + 0.01*(W**2).mean()
print(loss.item())

2.6418673992156982
2.6418673992156982


# EXERCISE 6

In [ ]:
import torch
dat = open("collection.txt","r").read().splitlines()
data = dat[1:]
chars = sorted(list(set('.'.join(data))))
chars.insert(24,'x')

In [ ]:
split = int(0.85*len(dat))
t_data = data[:split]
d_data = data[split:]

In [ ]:
xs1, xs2, xs_=[],[],[]
ys1, ys2, ys_=[],[],[]

for w in t_data[:3]:
    chs = ['.'] + list(w) + ['.']
    for ch1, ch2, ch3 in zip(chs, chs[1:], chs[2:]):
        xs1.append(bitoi[(ch1, ch2)])
        ys1.append(stoi[ch3])

for w in t_data[:3]:
  chs_ = ['.'] + list(reversed(w)) + ['.']
  for c1,c2,c3 in zip(chs_, chs_[1:], chs_[2:]):
    xs2.append(bitoi[(c1,c2)])
    ys2.append(stoi[c3])

for w in d_data:
  chsd = ['.'] +list(w) + ['.']
  for c1,c2,c3 in zip(chsd,chsd[1:],chsd[2:]):
    xs_.append(bitoi[(c1,c2)])
    ys_.append(stoi[c3])

In [ ]:
xs1 = torch.tensor(xs1)
ys1 = torch.tensor(ys1)
xs2 = torch.tensor(xs2)
ys2 = torch.tensor(ys2)
xs_ = torch.tensor(xs_)
ys_ = torch.tensor(ys_)

In [ ]:
#for k in range(10):
def cal_b_loss(xs1,ys1):
  num1 = xs1.nelement()
  logits1 = W[xs1]
  counts1 = logits1.exp()
  probs1 = counts1/counts1.sum(1,keepdims = True)
  loss1 = -probs1[torch.arange(num1),ys1].log().mean() + 0.01*(W**2).mean()
  print(loss1.item())

  W.grad = None
  loss1.backward()
  W.data += -50*W.grad

In [ ]:
cal_b_loss(xs1,ys1)

3.18754506111145


In [ ]:
question = input()
before = []
after=[]
i,j=0,0
for q in list(question):
  if(q=='_'):
    for x in range(2):
      i-=1
      if (i>=0):
        before.insert(0,list(question)[i])
      else:
        before.insert(0,'.')

    for x in range(2):
      j+=1
      if(j<len(list(question))):
        after.insert(0,list(question)[j])
      else:
        after.insert(0,'.')
  i+=1
  j+=1

ix1 = bitoi[tuple(before)]
ix2 = bitoi[tuple(after)]
ix = 0
while(ix==0):
 xenc1 = F.one_hot(torch.tensor([ix1]), num_classes=729).float()
 logits1 = xenc1 @ W
 counts1 = logits1.exp()
 probs1 = counts1/counts1.sum(1,keepdims =True)

 xenc2 = F.one_hot(torch.tensor([ix2]), num_classes =729).float()
 logits2 = xenc2 @ W
 counts2 = logits2.exp()
 probs2 = counts2/counts2.sum(1,keepdims = True)

 prob = probs1*probs2
 ix = torch.argmax(prob)

ans = question.replace('_',itos[ix.item()])

print(ans)